In [0]:
%python
df = spark.read.csv("/Volumes/workspace/project/project_files/netflix_titles.csv", header=True, inferSchema=True)

df.display()

In [0]:
%python

# save as managed table
df.write.mode("overwrite").saveAsTable("workspace.project.netflix_titles")

In [0]:
%python
from pyspark.sql import functions as F

table_name = "workspace.project.netflix_titles"
df = spark.table(table_name)

null_counts = df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c + "_nulls") 
    for c in df.columns
])

null_counts.display()

In [0]:
-- checking the data in sql 
SELECT * from workspace.project.netflix_titles
where show_id = 's5023'

In [0]:

--  Checking duplicates 

SELECT show_id, count(*) from workspace.project.netflix_titles
group by show_id
HAVING count(*) > 1

In [0]:
Select * from 
(SELECT upper(title),type, count(*) from workspace.project.netflix_titles
group by upper(title), type
HAVING count(*) > 1
)
-- SELECT * from workspace.project.netflix_titles
-- where title = 'DEATH NOTE'


In [0]:
with base as (SELECT *, 
row_number() OVER (partition by upper(title), type order by show_id) as rn 
FROM workspace.project.netflix_titles
)
Select * from base where rn > 1

In [0]:
-- Splitting the directors

Create or replace view workspace.project.netflix_directors 
AS 
SELECT
  show_id,
  trim(director_name) AS director
FROM (
  SELECT
    show_id,
    explode(split(director, ',')) AS director_name
  FROM workspace.project.netflix_titles
)

In [0]:
Select * from workspace.project.netflix_directors

In [0]:
-- Splitting the country

Create or replace view workspace.project.netflix_country 
AS 
SELECT
  show_id,
  trim(country) AS country
FROM (
  SELECT
    show_id,
    explode(split(country, ',')) AS country
  FROM workspace.project.netflix_titles
)

In [0]:
SELECT * from workspace.project.netflix_country 

In [0]:
-- Splitting the cast

Create or replace view workspace.project.netflix_cast 
AS 
SELECT
  show_id,
  trim(star_cast) as star_cats
FROM (
  SELECT
    show_id,
    explode(split(`cast`, ',')) AS star_cast
  FROM workspace.project.netflix_titles
)

In [0]:
SELECT * from workspace.project.netflix_cast

In [0]:
--  Splitting the Cast 

Create or replace view workspace.project.netflix_genre
AS 
SELECT
  show_id,
  trim(genre) as genre
FROM (
  SELECT
    show_id,
    explode(split(listed_in, ',')) AS genre
  FROM workspace.project.netflix_titles
)

In [0]:
SELECT * from workspace.project.netflix_genre

In [0]:
-- checking the datatype of date added 

DESC workspace.project.netflix_titles

In [0]:
with base as (SELECT *, 
row_number() OVER (partition by upper(title), type order by show_id) as rn 
FROM workspace.project.netflix_titles
)
Select show_id, type, title, cast(date_added as date) as date_added, release_year,rating, duration, description from base where rn > 1

In [0]:
SELECT country from workspace.project.netflix_country
where show_id = 's1001'